In [ ]:
import os
from pydub import AudioSegment
import pandas
import numpy
import requests
from IPython.display import clear_output

In [ ]:
birds = pandas.read_csv('birds_dataset.csv', index_col=0)
size_before = birds.shape[0]

In [ ]:
birds = birds[birds['duration'] >= 20][birds['name'] != 'Identity unknown']
size_after = birds.shape[0]

In [ ]:
print('Size of dataset before: {}'.format(size_before))
print('Size of dataset after: {}'.format(size_after))

In [ ]:
birds.head()

In [ ]:
genus_vs_file_path = pandas.DataFrame(columns=['file_id', 'genus', 'file_path'])

In [ ]:
def convert(src_file_name, dst_file_name, dest_folder='audio\\'):
    dst_file_name = dst_file_name + '.wav'
    sound = AudioSegment.from_mp3(src_file_name)[:20000]
    sound.export(dest_folder + dst_file_name, format="wav")
    print('converted ' + dst_file_name)
    return dest_folder + dst_file_name

In [ ]:
error_list = []
for index, row in birds.iterrows():
    try:
        clear_output(wait=True)
        r = requests.get(row.download_url, allow_redirects=True)
        with open(str(row.file_id)+'.mp3','wb') as op:
          op.write(r.content)
        dst_file_path = convert(str(row.file_id)+'.mp3', str(row.file_id))
        os.remove(str(row.file_id)+'.mp3')
        genus_vs_file_path = genus_vs_file_path.append({
                'file_id': row['file_id'],
                'genus': row['genus'],
                'file_path': dst_file_path
            }, ignore_index=True)
        print('Download and conversion in finished for {} of {} files'.format(index+1, size_after))
    except Exception as e:
        error_list.append(row.download_url)
        print(e)

In [ ]:
genus_vs_file_path.to_csv('genus_vs_file_path.csv')